In [9]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="qft",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------",qc.name, "with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------qft with qubit 3 ---------------
---------------qft with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7059b5d64e30>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x705926644c80>)]


In [10]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'010': 114, '000': 130, '100': 142, '110': 125, '101': 118, '001': 122, '111': 120, '011': 129}, {'000': 0.12499999999999986, '001': 0.1249999999999999, '010': 0.12500000000000003, '011': 0.12499999999999997, '100': 0.12499999999999994, '101': 0.12499999999999986, '110': 0.12499999999999997, '111': 0.12500000000000003})
(4, {'1101': 54, '1110': 53, '1011': 60, '0011': 86, '0111': 53, '1001': 59, '0101': 71, '0001': 62, '1111': 64, '1010': 68, '0000': 53, '0010': 65, '0100': 66, '1100': 48, '0110': 72, '1000': 66}, {'0000': 0.06249999999999989, '0001': 0.06249999999999992, '0010': 0.06249999999999989, '0011': 0.06249999999999989, '0100': 0.062499999999999944, '0101': 0.06249999999999993, '0110': 0.0624999999999999, '0111': 0.06249999999999986, '1000': 0.062499999999999944, '1001': 0.06249999999999993, '1010': 0.062499999999999944, '1011': 0.06249999999999992, '1100': 0.06249999999999992, '1101': 0.062499999999999944, '1110': 0.062499999999999875, '1111': 0.062499

In [11]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [13]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (qft, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (qft, 3 qubits): 0.9990
Hellinger Fidelity (qft, 4 qubits): 0.9948
